In [1]:
import pandas as pd
import numpy as np
import torch

In [2]:
prediction_value = torch.load('prediction_value.pt')
true_value = torch.load('test_future_value_tensor.pt')

In [3]:
prediction_value.shape,true_value.shape

(torch.Size([38949, 10, 4]), torch.Size([38949, 10, 4]))

In [4]:
def sMAPE_per_feature(prediction, true):
    # Calculate the absolute differences
    diff = torch.abs(prediction - true)
    
    # Calculate the sum of absolute values of the predictions and true values
    denom = (torch.abs(prediction) + torch.abs(true)) / 2
    
    # Avoid division by zero by adding a small epsilon to the denominator
    epsilon = 1e-8
    smape = 100 * (diff / (denom + epsilon))
    
    # Calculate the mean sMAPE for each feature (axis 2 is input_size)
    smape_per_feature = torch.mean(smape, dim=(0, 1))
    
    return smape_per_feature


In [5]:
# Now calculate sMAPE on the original values
smape_per_feature = sMAPE_per_feature(prediction_value,true_value)
print("sMAPE per feature:", smape_per_feature)

sMAPE per feature: tensor([148.5016, 144.2690, 146.6783, 177.8588])


In [6]:
# compile predicting result
ID = np.load('ID.npy', allow_pickle=True)

In [7]:
ID.shape

(38949, 10, 1)

In [8]:
# Load your tensor
time = torch.load('test_future_time_features_tensor.pt')

# Round to two decimal places
time = torch.round(time,decimals=2)

In [9]:
time.shape

torch.Size([38949, 10, 3])

In [10]:
# Convert PyTorch tensors to NumPy arrays
time_np = time.numpy()
time_np = np.round(time_np,2)
prediction_value_np = prediction_value.numpy()
prediction_value_np	 = np.round(prediction_value_np,2)
# Concatenate the arrays along axis 2 (last axis)
compiled_array = np.concatenate((ID, time_np, prediction_value_np), axis=2)

# Check the shape of the result
print(compiled_array.shape)


(38949, 10, 8)


In [11]:
np.save('prediction.npy', compiled_array)